In [0]:
 print('hellow spark')

hellow spark

In [0]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col,lit,udf
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.sql.functions import sum,avg,max,min,mean,count
spark=SparkSession.builder.appName('Spark dataframe').getOrCreate()


In [0]:
df=spark.read.options(header='True',inferSchema='True').csv('/FileStore/tables/OfficeData.csv')

In [0]:
df.show()

+-------------+----------+-----+------+---+-----+
employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
 James| Sales| NY| 90000| 34|10000|
 Michael| Sales| NY| 86000| 56|20000|
 Robert| Sales| CA| 81000| 30|23000|
 Maria| Finance| CA| 90000| 24|23000|
 Raman| Finance| CA| 99000| 40|24000|
 Scott| Finance| NY| 83000| 36|19000|
 Jen| Finance| NY| 79000| 53|15000|
 Jeff| Marketing| CA| 80000| 25|18000|
 Kumar| Marketing| NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+

In [0]:
df.groupBy('department').sum().show()

+----------+-----------+--------+----------+
department|sum(salary)|sum(age)|sum(bonus)|
+----------+-----------+--------+----------+
 Sales| 257000| 120| 53000|
 Finance| 351000| 153| 81000|
 Marketing| 171000| 75| 39000|
+----------+-----------+--------+----------+

In [0]:
# you can use multiple aggregate function like min ,max, sum, average,using aggregate function
df.groupBy('department').agg(sum('salary'),min('salary'),mean('bonus')).show()

+----------+-----------+-----------+------------------+
department|sum(salary)|min(salary)| avg(bonus)|
+----------+-----------+-----------+------------------+
 Sales| 257000| 81000|17666.666666666668|
 Finance| 351000| 79000| 20250.0|
 Marketing| 171000| 80000| 19500.0|
+----------+-----------+-----------+------------------+

In [0]:
df.groupBy('department','salary').agg(sum('salary'),min('salary'),mean('bonus')).show()

+----------+------+-----------+-----------+----------+
department|salary|sum(salary)|min(salary)|avg(bonus)|
+----------+------+-----------+-----------+----------+
 Finance| 99000| 99000| 99000| 24000.0|
 Marketing| 80000| 80000| 80000| 18000.0|
 Sales| 90000| 90000| 90000| 10000.0|
 Sales| 81000| 81000| 81000| 23000.0|
 Finance| 83000| 83000| 83000| 19000.0|
 Sales| 86000| 86000| 86000| 20000.0|
 Finance| 90000| 90000| 90000| 23000.0|
 Marketing| 91000| 91000| 91000| 21000.0|
 Finance| 79000| 79000| 79000| 15000.0|
+----------+------+-----------+-----------+----------+

In [0]:
# combining groupby and filtering 
# df2=df.filter(df.gender=='male').count().show()

In [0]:
# # #let us what are the user defined datatypes also called as udf
# #  we will see how udf works in this code snippet
# you have to create a basic python function and you can pass that function to the udf and you can use that function to perform operation you want to perform


# --------> in this column you have to create a new column called total_salary_udf and you have to add the salary and bonus column and print the output inside the new column



def get_total_salary(salary,bonus):
  return salary + bonus+ 122

total_salaryUDF=udf(lambda x,y : get_total_salary(x,y),IntegerType())

df.withColumn('total_salary',total_salaryUDF(df.salary,df.bonus)).show()

+-------------+----------+-----+------+---+-----+------------+
employee_name|department|state|salary|age|bonus|total_salary|
+-------------+----------+-----+------+---+-----+------------+
 James| Sales| NY| 90000| 34|10000| 100122|
 Michael| Sales| NY| 86000| 56|20000| 106122|
 Robert| Sales| CA| 81000| 30|23000| 104122|
 Maria| Finance| CA| 90000| 24|23000| 113122|
 Raman| Finance| CA| 99000| 40|24000| 123122|
 Scott| Finance| NY| 83000| 36|19000| 102122|
 Jen| Finance| NY| 79000| 53|15000| 94122|
 Jeff| Marketing| CA| 80000| 25|18000| 98122|
 Kumar| Marketing| NY| 91000| 50|21000| 112122|
+-------------+----------+-----+------+---+-----+------------+

In [0]:
# quiz 
# create a new column  called increment 
# if the employee is in 'NY' state increment his salary by 10% and bonus will be 5%
# if the employee is in the 'CA' state increment his salary wiith 12 % and bonus with 3% 


def increment(state,salary ,bonus):
  if state=='NY':
    sum=salary*0.10
    sum+=bonus*0.05
  elif state=='CA':
    sum=salary*0.12
    sum+=bonus*0.03
  return sum


increseUDF=udf(lambda x,y,z:increment(x,y,z),DoubleType())

df.withColumn('incrementbycity ',increseUDF(df.state,df.salary,df.bonus)).show()
   

+-------------+----------+-----+------+---+-----+----------------+
employee_name|department|state|salary|age|bonus|incrementbycity |
+-------------+----------+-----+------+---+-----+----------------+
 James| Sales| NY| 90000| 34|10000| 9500.0|
 Michael| Sales| NY| 86000| 56|20000| 9600.0|
 Robert| Sales| CA| 81000| 30|23000| 10410.0|
 Maria| Finance| CA| 90000| 24|23000| 11490.0|
 Raman| Finance| CA| 99000| 40|24000| 12600.0|
 Scott| Finance| NY| 83000| 36|19000| 9250.0|
 Jen| Finance| NY| 79000| 53|15000| 8650.0|
 Jeff| Marketing| CA| 80000| 25|18000| 10140.0|
 Kumar| Marketing| NY| 91000| 50|21000| 10150.0|
+-------------+----------+-----+------+---+-----+----------------+

In [0]:
# # cache and persist

# # now we can see how cache and persist works

# spark is lazy evaluator when we perform any action it will do all the computation from start instead of it we can use cache to speed up our actions


In [0]:
# #convert the dataframe to RDD
# rdd=df.rdd
# type(rdd)
# rdd.filter(lambda x :x[0]==28).collect()